<a href="https://colab.research.google.com/github/gustavofurini/cache-algorithm-simulation/blob/main/multiespecialista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import random

class QuadroNegro:
    def __init__(self):
        self.especialistas = []
        self.estadoCompartilhado = {
            'instancias-de-problemas': {
                'orcamento': [0],
                'desenho': [],
                'andamento': [],
                'instalacao_eletrica': [],
                'progresso': []
            },
            'contribuicoes': [],
            'progresso': 0
        }

    def adicionaEspecialista(self, especialista):
        self.especialistas.append(especialista)

    def adicionaContribuicao(self, contribuicao):
        self.estadoCompartilhado['contribuicoes'] += contribuicao

    def atualizaProgresso(self, progresso):
        self.estadoCompartilhado['progresso'] += progresso

    def adicionaTarefa(self, tarefa, parametros):
        self.estadoCompartilhado['instancias-de-problemas'][tarefa] = parametros

    def pegaTarefa(self, tarefa):
        return self.estadoCompartilhado['instancias-de-problemas'][tarefa]

    def mostraTarefas(self):
        print('Instâncias de problemas:', self.estadoCompartilhado['instancias-de-problemas'])

class Controlador:
    def __init__(self, quadro_negro, limite=100):
        self.quadro_negro = quadro_negro
        self.limite = limite

    def loop(self):
        while self.quadro_negro.estadoCompartilhado['progresso'] < self.limite:
            self.quadro_negro.mostraTarefas()
            for especialista in self.quadro_negro.especialistas:
                if especialista.eh_ativado:
                    especialista.contribui()
        return self.quadro_negro.estadoCompartilhado['contribuicoes']

import abc

class AbstractEspecialista:
    __metaclass__ = abc.ABCMeta

    def __init__(self, quadro_negro):
        self.quadro_negro = quadro_negro

    @property
    @abc.abstractmethod
    def eh_ativado(self):
        pass

    @property
    @abc.abstractmethod
    def expertise(self):
        pass

    @property
    def progresso(self):
        return random.randint(5, 25)

    @abc.abstractmethod
    def contribui(self):
        pass

# 1. Engenheiro Civil
class EngenheiroCivil(AbstractEspecialista):
    @property
    def eh_ativado(self):
        return True

    @property
    def expertise(self):
        orcamento = random.randint(100000, 600000)
        if orcamento > 400000:
            porte = 'grande'
        elif orcamento > 200000:
            porte = 'medio'
        else:
            porte = 'pequeno'
        return ['orcamento', {'valor': orcamento, 'porte': porte}]

    def contribui(self):
        contribuicao = [self.__class__.__name__, self.expertise]
        self.quadro_negro.adicionaContribuicao([contribuicao])
        self.quadro_negro.adicionaTarefa('orcamento', [contribuicao[1][1]])
        self.quadro_negro.atualizaProgresso(self.progresso)

# 2. Arquiteto
class Arquiteto(AbstractEspecialista):
    @property
    def eh_ativado(self):
        return isinstance(self.quadro_negro.pegaTarefa('orcamento')[0], dict)

    @property
    def expertise(self):
        orcamento_info = self.quadro_negro.pegaTarefa('orcamento')[0]
        porte = orcamento_info['porte']
        planta = f'Planta {porte} para orçamento R$ {orcamento_info["valor"]}'
        return ['desenho', {'planta': planta, 'porte': porte}]

    def contribui(self):
        contribuicao = [self.__class__.__name__, self.expertise]
        self.quadro_negro.adicionaContribuicao([contribuicao])
        self.quadro_negro.adicionaTarefa('desenho', [contribuicao[1][1]])
        self.quadro_negro.atualizaProgresso(self.progresso)

# 3. Pedreiro
class Pedreiro(AbstractEspecialista):
    @property
    def eh_ativado(self):
        return bool(self.quadro_negro.pegaTarefa('desenho'))

    @property
    def expertise(self):
        porte = self.quadro_negro.pegaTarefa('desenho')[0]['porte']
        if porte == 'pequeno':
            andamento = 'Obra em andamento rápido'
        elif porte == 'medio':
            andamento = 'Obra em andamento moderado'
        else:
            andamento = 'Obra em andamento lento'
        return ['andamento', {'descricao': andamento, 'porte': porte}]

    def contribui(self):
        contribuicao = [self.__class__.__name__, self.expertise]
        self.quadro_negro.adicionaContribuicao([contribuicao])
        self.quadro_negro.adicionaTarefa('andamento', [contribuicao[1][1]])
        self.quadro_negro.atualizaProgresso(self.progresso)

# 4. Eletricista
class Eletricista(AbstractEspecialista):
    @property
    def eh_ativado(self):
        return bool(self.quadro_negro.pegaTarefa('desenho'))

    @property
    def expertise(self):
        porte = self.quadro_negro.pegaTarefa('desenho')[0]['porte']
        if porte == 'pequeno':
            instalacao = 'Instalação simples'
        elif porte == 'medio':
            instalacao = 'Instalação padrão'
        else:
            instalacao = 'Instalação complexa'
        return ['instalacao_eletrica', {'descricao': instalacao, 'porte': porte}]

    def contribui(self):
        contribuicao = [self.__class__.__name__, self.expertise]
        self.quadro_negro.adicionaContribuicao([contribuicao])
        self.quadro_negro.adicionaTarefa('instalacao_eletrica', [contribuicao[1][1]])
        self.quadro_negro.atualizaProgresso(self.progresso)

# 5. Fiscal de Obras
class FiscalDeObras(AbstractEspecialista):
    @property
    def eh_ativado(self):
        return (
            bool(self.quadro_negro.pegaTarefa('andamento')) and
            bool(self.quadro_negro.pegaTarefa('instalacao_eletrica'))
        )

    @property
    def expertise(self):
        andamento_info = self.quadro_negro.pegaTarefa('andamento')[0]
        porte = andamento_info['porte']

        if porte == 'pequeno':
            progresso = random.randint(60, 100)
        elif porte == 'medio':
            progresso = random.randint(40, 70)
        else:
            progresso = random.randint(20, 50)

        return ['progresso', f'Progresso da obra ({porte}): {progresso}%']

    def contribui(self):
        contribuicao = [self.__class__.__name__, self.expertise]
        self.quadro_negro.adicionaContribuicao([contribuicao])
        self.quadro_negro.adicionaTarefa('progresso', [contribuicao[1][1]])
        self.quadro_negro.atualizaProgresso(self.progresso)

if __name__ == '__main__':
    quadro_negro = QuadroNegro()
    quadro_negro.adicionaEspecialista(EngenheiroCivil(quadro_negro))
    quadro_negro.adicionaEspecialista(Arquiteto(quadro_negro))
    quadro_negro.adicionaEspecialista(Pedreiro(quadro_negro))
    quadro_negro.adicionaEspecialista(Eletricista(quadro_negro))
    quadro_negro.adicionaEspecialista(FiscalDeObras(quadro_negro))

    contribuicoes = Controlador(quadro_negro, limite=25).loop()

    for c in contribuicoes:
        print(c)

Instâncias de problemas: {'orcamento': [0], 'desenho': [], 'andamento': [], 'instalacao_eletrica': [], 'progresso': []}
['EngenheiroCivil', ['orcamento', {'valor': 322638, 'porte': 'medio'}]]
['Arquiteto', ['desenho', {'planta': 'Planta medio para orçamento R$ 322638', 'porte': 'medio'}]]
['Pedreiro', ['andamento', {'descricao': 'Obra em andamento moderado', 'porte': 'medio'}]]
['Eletricista', ['instalacao_eletrica', {'descricao': 'Instalação padrão', 'porte': 'medio'}]]
['FiscalDeObras', ['progresso', 'Progresso da obra (medio): 59%']]
